In [1]:
import py_vollib
from py_vollib.black_scholes import black_scholes as bs
from py_vollib.black_scholes.implied_volatility import implied_volatility as iv
from py_vollib.black_scholes.greeks.analytical import delta 
from py_vollib.black_scholes.greeks.analytical import gamma
from py_vollib.black_scholes.greeks.analytical import rho
from py_vollib.black_scholes.greeks.analytical import theta
from py_vollib.black_scholes.greeks.analytical import vega

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
from tqdm import tqdm
from statsmodels.tsa.api import SimpleExpSmoothing

### Load traded options transaction data on DTCC for February 2021

In [3]:
df = pd.read_csv('database_TB.csv', index_col='Date')

### Load USD/AUD spot rates data from Yahoo Finance for February 2021

In [4]:
df2 = pd.read_csv('AUDUSD=X.csv', index_col='Date')

### Inner join two datasets on Date

In [5]:
df3 = df.join(df2)

### Calculate Implied Volatility

In [6]:
df4 = df3.reset_index()

# calculate the adjusted premium for FX option
df4['Adj_Prem'] = df4['Option Premium Amount'] / df4['USD_notional']

# calculate time to maturity as of execution date
df4['Date'] = pd.to_datetime(df4['Date']).dt.date
df4['Expiration Date'] = pd.to_datetime(df4['Expiration Date']).dt.date
df4['Maturity'] = (df4['Expiration Date'] - df4['Date']).dt.days / 365

In [7]:
# calculate Implied Volatility
Rf = 0.0001 - 0.0008
for i in tqdm(df4.index):
        df4.loc[i,'IV'] = iv(df4['Adj_Prem'][i], df4['Spot_USDAUD'][i], df4['Strike_USDAUD'][i], 
                 df4['Maturity'][i], Rf, df4['Type'][i])

100%|██████████| 807/807 [00:00<00:00, 2253.53it/s]


In [8]:
# drop useless columns
df5=df4.drop(['Open','High','Low','Close','Adj Close','Volume'], axis=1)

### Calculate Greeks

In [9]:
for i in tqdm(df5.index):
        df5.loc[i,'Delta'] = delta(df5['Type'][i],df5['Spot_USDAUD'][i],df5['Strike_USDAUD'][i],df5['Maturity'][i],Rf,df5['IV'][i])
        df5.loc[i,'Gamma'] = gamma(df5['Type'][i],df5['Spot_USDAUD'][i],df5['Strike_USDAUD'][i],df5['Maturity'][i],Rf,df5['IV'][i])
        df5.loc[i,'Rho'] = rho(df5['Type'][i],df5['Spot_USDAUD'][i],df5['Strike_USDAUD'][i],df5['Maturity'][i],Rf,df5['IV'][i])
        df5.loc[i,'Theta'] = theta(df5['Type'][i],df5['Spot_USDAUD'][i],df5['Strike_USDAUD'][i],df5['Maturity'][i],Rf,df5['IV'][i])
        df5.loc[i,'Vega'] = vega(df5['Type'][i],df5['Spot_USDAUD'][i],df5['Strike_USDAUD'][i],df5['Maturity'][i],Rf,df5['IV'][i])

100%|██████████| 807/807 [00:00<00:00, 807.12it/s]


### Export the data to "database_TB_Ready.csv" for Tableau dashboard

In [10]:
df5.to_csv('database_TB_Ready.csv', index=False)